### Workflow

1. Per la prima stringa si suggeriscono solo le skill che iniziano per la stringa inserita fino a quel momento.\
TODO: decidere quali mostrare, quelle più lunghe? a caso?


2. Per le successive stringe si considerano le skill inserite fino a quel momento: partendo dalla stringa inserita si ricavano le skill che contengono quella stringa, come in 1, poi per ognuna di queste si calcola la similarità del coseno con le skill già inserite e si suggerisce la skill con la media delle similarità più altra\
TODO: decidere quante skill mostrare. Usare la gerarchia delle skill come altra metrica di suggerimento, ad esempio in base alla distanza dal primo antenato in comune


3. suggerire anche una skill che non contenga la stringa inserita fino a quel momento ma che sia semplicemente la più simile con quelle inserite precedentemente, calcolata come per 2


4. Gestire gli errori di typo: individuarli e suggerire un'alternativa

In [143]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from bisect import bisect_left
import ipywidgets as widgets
import difflib
from gensim.models import FastText, WordEmbeddingSimilarityIndex
from IPython.core.display import display, HTML
from gensim.similarities import SparseTermSimilarityMatrix, MatrixSimilarity
from gensim.matutils import Dense2Corpus

In [152]:
t = SuggestsTextArea(vectors)
t.show_form()

In [151]:
class SuggestsTextArea:
    
    def __init__(self, model):
        # peso dell'input inserito nel calcolo della similarità finale
        self.alpha = 0.8
        
        self.skills_list = pd.read_excel("data/2020_06_09 Allocation to ONET.xlsx")['escoskill_level_3']
        self.vectors = model #FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")
        #self.batch = np.array(list(self.vectors.values()))[0:100]
        #self.cos_sim_matrix = pd.DataFrame(cosine_similarity(self.batch), 
        #                                   columns=list(self.vectors.keys())[0:100],
        #                                   index=list(self.vectors.keys())[0:100])
        
        # lista dei bottoni
        self.suggest_buttons = list()
        self.suggest_buttons_context = list()
        
        # set up della text area
        self.textArea = widgets.Textarea(
            value='',
            placeholder='Type something',
            disabled=False,
            tooltip='Enter the name of the Text field',
            height='90px',
            layout=widgets.Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
        )
        
        # set up della box per i bottoni dei suggerimenti per l'input
        self.suggests = widgets.HBox(self.suggest_buttons)
        
        # set up della box per i bottoni dei suggerimenti solo per il context
        self.suggests_context = widgets.HBox(self.suggest_buttons_context)
        
        # set up dell'intero form
        self.form = widgets.VBox([widgets.VBox([widgets.Label(value='Skills:'), self.textArea]), 
                                 widgets.HBox([widgets.Label(value='Suggests:'), self.suggests]),
                                 widgets.HBox([widgets.Label(value='Did you also mean...?:'), 
                                               self.suggests_context])])
    
    
    def find_sub_sting(self, word, words_list, limit=4):
        '''
        Find all skill that contain the new string insert by user
        '''
        return sorted([(w , self.vectors.wv.similarity(word, w)) for w in words_list], key=lambda tup: tup[1], 
                      reverse=True)[:limit]
        #return [w for w in sorted(words_list) if w.lower().startswith(word.lower())]
    
    
    def get_best_similarity_skill(self, context, new_input):
        '''
        Get the four most similarity skills respect of the user's input
        '''
        
        # similarità tra input e lista delle skill
        find_suggests = self.find_sub_sting(new_input, self.skills_list, limit=4)
        res = dict()
        res_sim = dict()
        
        # calcolo delle medie delle similarità tra le skill candidate come suggerimenti e le skill già inserite
        for skill in [s[0] for s in find_suggests]:
            res[skill] = np.array([self.vectors.wv.similarity(skill, c) for c in context]).mean()
        
        # calcolo della similarità aggregata
        for i in range(0, 4):
            skill = find_suggests[i][0]
            final_sim = (self.alpha * find_suggests[i][1]) + ((1-self.alpha) * res[skill])
            res_sim[skill] = final_sim
        return {k: v for k, v in sorted(res_sim.items(), key=lambda item: item[1], reverse=True)}
        #(a * sim(parola_corrente, parole_excel) + (1-a) * media(similarità(parola_precedente, parole, excel)))
        #candidate_skill = self.cos_sim_matrix.loc[context, find_suggests]
        #return candidate_skill.mean(axis = 0).sort_values(ascending=False)
        #return candidate_skill.mean(axis = 0).idxmax(), candidate_skill.mean(axis = 0).max()
        
    
    def show_form(self):
        '''
        Show the text area widget
        '''
        display(self.form)
        self.textArea.observe(self.suggests_manager, names='value')
    
    def suggests_manager(self, widget):
        '''
        Main function of the class, manage the suggestions in different case
        '''
        self.close_widgets()
        old_input = widget['old']
        new_input = widget['new'].split(' ')[-1]
        context = old_input.split(', ')[:-1]
        if context != []:
            sug = self.get_best_similarity_skill(context, new_input)
            for index, value in sug.items():
                b = widgets.Button(
                    description=f'{index} - {round(value*100, 2)}%', # fix negative similarity: e.g. -35%
                    disabled=False,
                    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip=f'{index} - {round(value*100, 2)}%'
                )

                b.on_click(self.add_skill)
                self.suggest_buttons.append(b)
                
            #display(widgets.HBox(self.suggest_buttons))
            
        else:
            if len(new_input)>3:
                find_suggests = self.find_sub_sting(new_input, self.skills_list, limit=4)                    
                    
                for sug in find_suggests:

                    b = widgets.Button(
                        description=f'{sug[0]} - {round(sug[1]*100, 2)}%',
                        disabled=False,
                        button_style='', # 'success', 'info', 'warning', 'danger' or ''
                        tooltip=f'{sug[0]} - {round(sug[1]*100, 2)}%'
                    )

                    b.on_click(self.add_skill)
                    self.suggest_buttons.append(b)
        self.suggests.children=tuple(self.suggest_buttons)
                

    def add_skill(self, btn_object):
        '''
        Button click handler, add the skill pressed to the text area
        '''
        skill_list = self.textArea.value.split(' ')
        skill_list[-1] = btn_object.description.split(" -")[0]
        new_contest = ' '.join(skill_list) + ', '
        self.textArea.value = new_contest
        self.close_widgets()
    
    
    def close_widgets(self):# fix button go down on every input
        '''
        Remove button of old suggestions
        '''
        for w in self.suggest_buttons:
            w.close()
        self.suggest_buttons = list()

    
    

In [4]:
vectors = FastText.load_fasttext_format("data/ft_vectors_cbow_50_10_0_05.bin")

/home/simone/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


In [156]:
index = MatrixSimilarity(Dense2Corpus(vectors.wv.vectors))

In [155]:
len(vectors.wv.vectors)

113656

In [157]:
similarity_matrix = cosine_similarity(vectors.wv.vectors)

MemoryError: 

In [135]:
termsim_index = WordEmbeddingSimilarityIndex(vectors.wv)
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, vectors.wv.vocab)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [71]:
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display

vb = wg.VBox([wg.Text('1'),wg.Text('2')])
btn = wg.Button(description = 'Add') 

def on_bttn_clicked(b):        
    vb.children=tuple(list(vb.children) + [wg.Text('3')]) 

btn.on_click(on_bttn_clicked)
display(vb, btn)

list(vb.children)

Button(description='Add', style=ButtonStyle())

[Text(value='1'), Text(value='2')]

In [89]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, HBox, VBox

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

b1 = widgets.Button(
    description=f'eeeee',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
b2 = widgets.Button(
    description=f'ewwwww',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)

box = VBox([b1, b2])

btn = wg.Button(description = 'Add') 
def on_bttn_clicked(btn):        
    box.layout.visibility = 'visible'

btn.on_click(on_bttn_clicked)

dele = wg.Button(description = 'del') 
def dee(btn):
    print("ss")
    box.layout.visibility = 'hidden'

dele.on_click(dee)

form_items = [
    Box([Label(value='Information'),
         Textarea()], layout=form_item_layout),
    Box([Label(value='Suggests'),
         box], layout=form_item_layout),
    Box([Label(value=''),
         btn], layout=form_item_layout),
    Box([Label(value=''),
         dele], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='100%'
))


display(form)

#resetButton.layout.visibility = 'hidden'

Box(children=(Box(children=(Label(value='Information'), Textarea(value='')), layout=Layout(display='flex', fle…

In [104]:
import ipywidgets as wg
from ipywidgets import Layout
from IPython.display import display

vb = wg.VBox([wg.Text('1'),wg.Text('2')])
btn = wg.Button(description = 'Add') 

def on_bttn_clicked(b):        
    vb.children=tuple(list(vb.children) + [wg.Text('3')]) 

btn.on_click(on_bttn_clicked)
display(vb, btn)

list(vb.children)

Button(description='Add', style=ButtonStyle())

[Text(value='1'), Text(value='2')]